In [47]:
import pandas as pd
# import pandas_market_calendars as mcal
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [48]:
# df6 = pd.read_csv("/content/drive/MyDrive/CSV檔/報酬率.csv")

# data = pd.read_excel('/content/drive/MyDrive/CSV檔/sofr.xlsx',parse_dates=['Start Date','Maturity'])
# data = data.rename(columns={'Unnamed: 4':'Tenor'})
# data = pd.read_excel('/content/drive/MyDrive/CSV檔/sofr.xlsx')
data = pd.read_excel('HW1_v2.xlsx', header = 1)
data.columns = [' ','DP/SW','Start Date','Generator','Maturity','Contract','DF','Zero Rate','Market Data','  ','TimeDelta','OD','TodayDelta','Prior','DF_Cum']
data
data['Market Data'] = data['Market Data']/100

#先處理O/N T/N部分

In [49]:
data.iloc[0:4,]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
0,NaN,Dp,2022-10-13,SOFRRATE,2022-10-14,O/N,0.999915,0.030922,0.030500,NaN,1,0,1,0,NaN
1,NaN,Dp,2022-10-14,SOFRRATE,2022-10-17,T/N,0.999661,0.030920,0.030500,NaN,3,1,4,3,NaN
2,NaN,Sw,2022-10-17,\USD OIS SOFR,2022-10-31,2W,0.998480,0.030846,0.030420,NaN,14,4,18,14,NaN
3,NaN,Sw,2022-10-17,\USD OIS SOFR,2022-11-17,1M,0.996827,0.033141,0.033016,NaN,31,4,35,17,NaN


In [50]:
data['DF'][0] = 1 / ( 1+ data['Market Data'][0] * data['TimeDelta'][0]/360 )
data['DF'][1] = (1 / ( 1+ data['Market Data'][1] * data['TimeDelta'][1]/360 ) )*data['DF'][0] #T/N
data['Zero Rate'][0] =  -np.log(data['DF'][0]) * 365/ data['TodayDelta'][0]
data['Zero Rate'][1] =  -np.log(data['DF'][1]) * 365/ data['TodayDelta'][1]

#處理2W到1Y


In [51]:
for i in range(2,11,1):

    data['DF'][i] = 1 / ( 1+ data['Market Data'][i] * data['TimeDelta'][i]/360 ) * data['DF'][1]
    data['Zero Rate'][i] =  -np.log(data['DF'][i]) * 365/ data['TodayDelta'][i]

In [52]:
up =  (data['DF'][1] - data['Market Data'][11]*(data['TimeDelta'][8]/360) * data['DF'][8])
down = (1+ data['Market Data'][11] * (data['TimeDelta'][11]-data['TimeDelta'][8])/360)
data['DF'][11] = up/down
data['Zero Rate'][11] =  -np.log(data['DF'][11]) * 365/ data['TodayDelta'][11]

#處理2Y到10Y

In [53]:
data[8:21]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
8,NaN,Sw,2022-10-17,\USD OIS SOFR,2023-04-17,6M,0.978857,0.041936,0.042041,NaN,182,4,186,31,NaN
9,NaN,Sw,2022-10-17,\USD OIS SOFR,2023-07-17,9M,0.967582,0.043424,0.043719,NaN,273,4,277,91,NaN
10,NaN,Sw,2022-10-17,\USD OIS SOFR,2023-10-17,1Y,0.956557,0.043934,0.044445,NaN,365,4,369,92,NaN
11,NaN,Sw,2022-10-17,\USD OIS SOFR,2024-04-17,18M,0.935862,0.043832,0.044111,NaN,548,4,552,183,NaN
12,NaN,Sw,2022-10-17,\USD OIS SOFR,2024-10-17,2Y,0.917297,0.042868,0.043294,NaN,731,4,735,366,0.969842
13,NaN,Sw,2022-10-17,\USD OIS SOFR,2025-10-17,3Y,0.884253,0.040818,0.041232,NaN,1096,4,1100,365,1.902428
14,NaN,Sw,2022-10-17,\USD OIS SOFR,2026-10-19,4Y,0.853610,0.039381,0.039805,NaN,1463,4,1467,367,2.798962
15,NaN,Sw,2022-10-17,\USD OIS SOFR,2027-10-18,5Y,0.824707,0.038419,0.038852,NaN,1827,4,1831,364,3.669170
16,NaN,Sw,2022-10-17,\USD OIS SOFR,2028-10-17,6Y,0.797038,0.037705,0.038150,NaN,2192,4,2196,365,4.503040
17,NaN,Sw,2022-10-17,\USD OIS SOFR,2029-10-17,7Y,0.770797,0.037103,0.037564,NaN,2557,4,2561,365,5.311148


In [54]:
def fun(c):
    up = data['DF'][1] - data['Market Data'][c]* data['DF_Cum'][c]
    down = (1+ data['Market Data'][c] * (data['Prior'][c])/360)
    data['DF'][c] = up/down
    return  data['DF'][c]


for i in range(12,22,1):
    if i == 12:  #2Yr部分
        data['DF_Cum'][i] =  data['DF'][10] * data['TimeDelta'][10]/360 

    else:
        data['DF_Cum'][i] =  data['DF_Cum'][i-1] + data['DF'][i-1] * data['Prior'][i-1]/360  #3Y
    
    data['DF'][i] = fun(i)
    data['Zero Rate'][i] =  -np.log(data['DF'][i]) * 365/ data['TodayDelta'][i]
   

In [55]:
data[20:24]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
20,NaN,Sw,2022-10-17,\USD OIS SOFR,2032-10-18,10Y,0.696793,0.036048,0.036535,NaN,3654,4,3658,367,7.579476
21,NaN,Sw,2022-10-17,\USD OIS SOFR,2033-10-17,11Y,NaN,NaN,NaN,NaN,4018,4,4022,364,8.289817
22,NaN,Sw,2022-10-17,\USD OIS SOFR,2034-10-17,12Y,0.651450,0.035656,0.036238,NaN,4383,4,4387,365,NaN
23,NaN,Sw,2022-10-17,\USD OIS SOFR,2035-10-17,13Y,NaN,NaN,NaN,NaN,4748,4,4752,365,NaN


#11y 12y 部分

In [56]:
from scipy.optimize import fsolve
import math


In [57]:
data[20:24]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
20,NaN,Sw,2022-10-17,\USD OIS SOFR,2032-10-18,10Y,0.696793,0.036048,0.036535,NaN,3654,4,3658,367,7.579476
21,NaN,Sw,2022-10-17,\USD OIS SOFR,2033-10-17,11Y,NaN,NaN,NaN,NaN,4018,4,4022,364,8.289817
22,NaN,Sw,2022-10-17,\USD OIS SOFR,2034-10-17,12Y,0.651450,0.035656,0.036238,NaN,4383,4,4387,365,NaN
23,NaN,Sw,2022-10-17,\USD OIS SOFR,2035-10-17,13Y,NaN,NaN,NaN,NaN,4748,4,4752,365,NaN


In [58]:
def func(i):

    x,y = i[0], i[1] 
    A = (data['Zero Rate'][20]*data['Prior'][22]+ y * data['Prior'][21])/(data['Prior'][21]+data['Prior'][22]) 
    A1 = A - x

    data['DF_Cum'][22] =  data['DF_Cum'][21] + math.exp(-1*x*data['TodayDelta'][21]/365) * data['Prior'][21]/360

    B = data['DF'][1] - (data['Market Data'][22] * data['DF_Cum'][22])
    C = 1 + (data['Market Data'][22])*(data['Prior'][22]/360)
    D = math.exp(y*(-1)* data['TodayDelta'][22]/365)
    E = (B/C)-D
    return A1,E


x,y = fsolve(func,[0,0])
 
print(x)
print(y)

data['Zero Rate'][21] = x 
data['Zero Rate'][22] = y
data['DF'][21] = math.exp(-x * data['TodayDelta'][21]/365)
data['DF'][22] = math.exp(-y * data['TodayDelta'][22]/365)
data['DF_Cum'][22] = data['DF_Cum'][21] + data['DF'][21] * data['Prior'][21]/360
data['DF_Cum'][23] = data['DF_Cum'][22] + data['DF'][22] * data['Prior'][22]/360



0.03590128780009734
0.03575445260964286


#13,14,15Yr

In [59]:
data[22:26]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
22,NaN,Sw,2022-10-17,\USD OIS SOFR,2034-10-17,12Y,0.650679,0.035754,0.036238,NaN,4383,4,4387,365,8.970572
23,NaN,Sw,2022-10-17,\USD OIS SOFR,2035-10-17,13Y,NaN,NaN,NaN,NaN,4748,4,4752,365,9.630288
24,NaN,Sw,2022-10-17,\USD OIS SOFR,2036-10-17,14Y,NaN,NaN,NaN,NaN,5114,4,5118,366,NaN
25,NaN,Sw,2022-10-17,\USD OIS SOFR,2037-10-19,15Y,0.590344,0.035073,0.035875,NaN,5481,4,5485,367,9.609004


In [60]:
def func(i):

    #內插zero rate
    time = data['Prior'][23]+data['Prior'][24]+data['Prior'][25]
    x,y,z = i[0], i[1], i[2]
    A1 = (data['Zero Rate'][22] * (data['Prior'][24]+data['Prior'][25]) + z * data['Prior'][23])/ time 
    A2 = (data['Zero Rate'][22] * data['Prior'][25]+ z * (data['Prior'][23]+data['Prior'][24]))/ time  


    A11 = A1 - x
    A22 = A2 - y

    data['DF_Cum'][24] =  data['DF_Cum'][23] + math.exp(-1 * x * data['TodayDelta'][23]/365) * data['Prior'][23]/360
    data['DF_Cum'][25] =  data['DF_Cum'][24] + math.exp(-1 * y * data['TodayDelta'][24]/365) * data['Prior'][24]/360

    B = data['DF'][1] - (data['Market Data'][25] * data['DF_Cum'][25])
    C = 1 + (data['Market Data'][25])*(data['Prior'][25]/360)
    D = math.exp(z*(-1)* data['TodayDelta'][25]/365)
    E = (B/C)-D
    return A11,A22,E


x,y,z = fsolve(func,[0,0,0])
 
print(x)
print(y)
print(z)

data['Zero Rate'][23] = x 
data['Zero Rate'][24] = y
data['Zero Rate'][25] = z 

data['DF'][23] = math.exp(-x * data['TodayDelta'][23]/365)
data['DF'][24] = math.exp(-y * data['TodayDelta'][24]/365)
data['DF'][25] = math.exp(-z * data['TodayDelta'][25]/365)
data['DF_Cum'][24] = data['DF_Cum'][23] + data['DF'][23] * data['Prior'][23]/360
data['DF_Cum'][25] = data['DF_Cum'][24] + data['DF'][24] * data['Prior'][24]/360
data['DF_Cum'][26] = data['DF_Cum'][25] + data['DF'][25] * data['Prior'][25]/360

0.03562934844202651
0.035503901523266004
0.035378111853361344


#16Yr-20Yr & 21Yr-25Yr & 26Yr-30Yr

In [61]:
data[25:31]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
25,NaN,Sw,2022-10-17,\USD OIS SOFR,2037-10-19,15Y,0.587640,0.035378,0.035875,NaN,5481,4,5485,367,10.885848
26,NaN,Sw,2022-10-17,\USD OIS SOFR,2038-10-18,16Y,NaN,NaN,NaN,NaN,5845,4,5849,364,11.484915
27,NaN,Sw,2022-10-17,\USD OIS SOFR,2039-10-17,17Y,NaN,NaN,NaN,NaN,6209,4,6213,364,NaN
28,NaN,Sw,2022-10-17,\USD OIS SOFR,2040-10-19,18Y,NaN,NaN,NaN,NaN,6577,4,6581,368,NaN
29,NaN,Sw,2022-10-17,\USD OIS SOFR,2041-10-17,19Y,NaN,NaN,NaN,NaN,6940,4,6944,363,NaN
30,NaN,Sw,2022-10-17,\USD OIS SOFR,2042-10-17,20Y,0.512986,0.033334,0.034741,NaN,7305,4,7309,365,11.409552


In [62]:
for j in range(0,3,1):  
    k = 5*j
    
    def func(i):
        #內插zero rate
        x,y,z,w,u = i[0], i[1], i[2], i[3], i[4]

        time = data['Prior'][26+k]+data['Prior'][27+k]+data['Prior'][28+k]+data['Prior'][29+k]+data['Prior'][30+k]

        A1 = (data['Zero Rate'][25+k] * (time - data['Prior'][26+k]) + u * data['Prior'][26+k])/ time 
        A2 = (data['Zero Rate'][25+k] * (time - data['Prior'][26+k] - data['Prior'][27+k])+ u * (data['Prior'][26+k]+data['Prior'][27+k]))/ time  
        A3 = (data['Zero Rate'][25+k] * (data['Prior'][29+k]+data['Prior'][30+k]) + u * (time - data['Prior'][29+k] - data['Prior'][30+k]))/ time    
        A4 = (data['Zero Rate'][25+k] * (data['Prior'][30+k]) + u * (time - data['Prior'][30+k]))/ time

        A11 = A1 - x
        A22 = A2 - y
        A33 = A3 - z
        A44 = A4 - w

        data['DF_Cum'][27+k] =  data['DF_Cum'][26+k] + math.exp(-1* x * data['TodayDelta'][26+k]/365) * data['Prior'][26+k]/360
        data['DF_Cum'][28+k] =  data['DF_Cum'][27+k] + math.exp(-1* y * data['TodayDelta'][27+k]/365) * data['Prior'][27+k]/360
        data['DF_Cum'][29+k] =  data['DF_Cum'][28+k] + math.exp(-1* z * data['TodayDelta'][28+k]/365) * data['Prior'][28+k]/360
        data['DF_Cum'][30+k] =  data['DF_Cum'][29+k] + math.exp(-1* w * data['TodayDelta'][29+k]/365) * data['Prior'][29+k]/360

        B = data['DF'][1] - (data['Market Data'][30+k] * data['DF_Cum'][30+k])
        C = 1 + (data['Market Data'][30+k])*(data['Prior'][30+k]/360)
        D = math.exp(u*(-1)* data['TodayDelta'][30+k]/365)
        E = (B/C)-D
        return A11,A22,A33,A44,E


    x,y,z,w,u = fsolve(func,[0,0,0,0,0])
    
    print(x)
    print(y)
    print(z)
    print(w)
    print(u)
    data['Zero Rate'][26+k] = x 
    data['Zero Rate'][27+k] = y
    data['Zero Rate'][28+k] = z 
    data['Zero Rate'][29+k] = w 
    data['Zero Rate'][30+k] = u 
    data['DF'][26+k] = math.exp(-x * data['TodayDelta'][26+k]/365)
    data['DF'][27+k] = math.exp(-y * data['TodayDelta'][27+k]/365)
    data['DF'][28+k] = math.exp(-z * data['TodayDelta'][28+k]/365)
    data['DF'][29+k] = math.exp(-w * data['TodayDelta'][29+k]/365)
    data['DF'][30+k] = math.exp(-u * data['TodayDelta'][30+k]/365)
    data['DF_Cum'][27+k] = data['DF_Cum'][26+k] + data['DF'][26+k] * data['Prior'][26+k]/360
    data['DF_Cum'][28+k] = data['DF_Cum'][27+k] + data['DF'][27+k] * data['Prior'][27+k]/360
    data['DF_Cum'][29+k] = data['DF_Cum'][28+k] + data['DF'][28+k] * data['Prior'][28+k]/360
    data['DF_Cum'][30+k] = data['DF_Cum'][29+k] + data['DF'][29+k] * data['Prior'][29+k]/360
    data['DF_Cum'][31+k] = data['DF_Cum'][30+k] + data['DF'][30+k] * data['Prior'][30+k]/360

0.03510464755598431
0.034831183258607276
0.034554713859061253
0.03428200083722647
0.034007785264307186
0.033578640079016245
0.03315300289251514
0.03272619637308408
0.03229938985365303
0.03187258333422198
0.03148845922324618
0.031108510374346213
0.03072856152544624
0.030347568861027314
0.02996553238108943


#31Yr-40Yr

In [63]:
data[40:51]

,,DP/SW,Start Date,Generator,Maturity,Contract,DF,Zero Rate,Market Data,,TimeDelta,OD,TodayDelta,Prior,DF_Cum
40,NaN,Sw,2022-10-17,\USD OIS SOFR,2052-10-17,30Y,0.406590,0.029966,0.031640,NaN,10958,4,10962,366,18.330998
41,NaN,Sw,2022-10-17,\USD OIS SOFR,2053-10-17,31Y,NaN,NaN,NaN,NaN,11323,4,11327,365,18.744364
42,NaN,Sw,2022-10-17,\USD OIS SOFR,2054-10-19,32Y,NaN,NaN,NaN,NaN,11690,4,11694,367,NaN
43,NaN,Sw,2022-10-17,\USD OIS SOFR,2055-10-18,33Y,NaN,NaN,NaN,NaN,12054,4,12058,364,NaN
44,NaN,Sw,2022-10-17,\USD OIS SOFR,2056-10-17,34Y,NaN,NaN,NaN,NaN,12419,4,12423,365,NaN
45,NaN,Sw,2022-10-17,\USD OIS SOFR,2057-10-17,35Y,NaN,NaN,NaN,NaN,12784,4,12788,365,NaN
46,NaN,Sw,2022-10-17,\USD OIS SOFR,2058-10-17,36Y,NaN,NaN,NaN,NaN,13149,4,13153,365,NaN
47,NaN,Sw,2022-10-17,\USD OIS SOFR,2059-10-17,37Y,NaN,NaN,NaN,NaN,13514,4,13518,365,NaN
48,NaN,Sw,2022-10-17,\USD OIS SOFR,2060-10-18,38Y,NaN,NaN,NaN,NaN,13881,4,13885,367,NaN
49,NaN,Sw,2022-10-17,\USD OIS SOFR,2061-10-17,39Y,NaN,NaN,NaN,NaN,14245,4,14249,364,NaN


In [64]:
def func(i):
    #內插zero rate
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10 = i[0], i[1], i[2], i[3], i[4],i[5], i[6], i[7], i[8], i[9]
    

    t1 = data['Prior'][41]
    t2 = data['Prior'][42]
    t3 = data['Prior'][43]
    t4 = data['Prior'][44]
    t5 = data['Prior'][45]
    t6 = data['Prior'][46]
    t7 = data['Prior'][47]
    t8 = data['Prior'][48]
    t9 = data['Prior'][49]
    t10 = data['Prior'][50]

    time = t1+t2+t3+t4+t5+t6+t7+t8+t9+t10

    A1 = (data['Zero Rate'][40] * (time - t1) + x10 * t1)/ time 
    A2 = (data['Zero Rate'][40] * (time - t1 - t2)+ x10 * (t1+t2))/ time  
    A3 = (data['Zero Rate'][40] * (time - t1 - t2 - t3) + x10 * (t1+t2+t3))/ time    
    A4 = (data['Zero Rate'][40] * (time - t1 - t2 - t3 - t4) + x10 * (t1+t2+t3+t4))/ time
    A5 = (data['Zero Rate'][40] * (time - t1 - t2 - t3 - t4 - t5) + x10 * (t1+t2+t3+t4+t5))/ time
    A6 = (data['Zero Rate'][40] * (t7+t8+t9+t10) + x10 * (time- t7 - t8 - t9 - t10))/ time
    A7 = (data['Zero Rate'][40] * (t8+t9+t10) + x10 * (time - t8 - t9 - t10))/ time
    A8 = (data['Zero Rate'][40] * (t9+t10) + x10 * (time - t9 - t10))/ time
    A9 = (data['Zero Rate'][40] * (t10) + x10 * (time - t10))/ time

    A11 = A1 - x1
    A22 = A2 - x2
    A33 = A3 - x3
    A44 = A4 - x4
    A55 = A5 - x5
    A66 = A6 - x6
    A77 = A7 - x7
    A88 = A8 - x8
    A99 = A9 - x9
   
    # lis = [x1,x2,x3,x4,x5,x6,x7,x8,x9,x10]
    # for li in range(0,9,1):
    data['DF_Cum'][42] = data['DF_Cum'][41] +math.exp(-1* x1 * data['TodayDelta'][41]/365) * data['Prior'][41]/360
    data['DF_Cum'][43] = data['DF_Cum'][42] +math.exp(-1* x2 * data['TodayDelta'][42]/365) * data['Prior'][42]/360
    data['DF_Cum'][44] = data['DF_Cum'][43] +math.exp(-1* x3 * data['TodayDelta'][43]/365) * data['Prior'][43]/360
    data['DF_Cum'][45] = data['DF_Cum'][44] +math.exp(-1* x4 * data['TodayDelta'][44]/365) * data['Prior'][44]/360
    data['DF_Cum'][46] = data['DF_Cum'][45] +math.exp(-1* x5 * data['TodayDelta'][45]/365) * data['Prior'][45]/360
    data['DF_Cum'][47] = data['DF_Cum'][46] +math.exp(-1* x6 * data['TodayDelta'][46]/365) * data['Prior'][46]/360
    data['DF_Cum'][48] = data['DF_Cum'][47] +math.exp(-1* x7 * data['TodayDelta'][47]/365) * data['Prior'][47]/360
    data['DF_Cum'][49] = data['DF_Cum'][48] +math.exp(-1* x8 * data['TodayDelta'][48]/365) * data['Prior'][48]/360
    data['DF_Cum'][50] = data['DF_Cum'][49] +math.exp(-1* x9 * data['TodayDelta'][49]/365) * data['Prior'][49]/360

    B = data['DF'][1] - (data['Market Data'][50] * data['DF_Cum'][50])
    C = 1 + (data['Market Data'][50])*(data['Prior'][50]/360)
    D = math.exp(x10*(-1)* data['TodayDelta'][50]/365)
    E = (B/C)-D
    return A11,A22,A33,A44,A55,A66,A77,A88,A99,E


x1,x2,x3,x4,x5,x6,x7,x8,x9,x10 = fsolve(func,[0,0,0,0,0,0,0,0,0,0])

print(x1)
print(x2)
print(x3)
print(x4)
print(x5)
print(x6)
print(x7)
print(x8)
print(x9)
print(x10)

lis = [x1,x2,x3,x4,x5,x6,x7,x8,x9,x10]

0.02962635486855909
0.029285318849110776
0.028947070590039425
0.028607893077509085
0.028268715564978744
0.027929538052448404
0.027590360539918066
0.027249324520469756
0.0269110762613984
0.02657189874886806


In [65]:
lis = [x1,x2,x3,x4,x5,x6,x7,x8,x9,x10]

In [66]:
for i in range(0,10,1):

    data['Zero Rate'][41+i] = lis[i]

    data['DF'][41+i] = math.exp(-lis[i] * data['TodayDelta'][41+i]/365)
    
    if i <9:
         data['DF_Cum'][42+i] = data['DF_Cum'][41] + data['DF'][41+i] * data['Prior'][41+i]/360




In [67]:
data.to_excel('111352036_林雅琪_1013.xlsx')